In [1]:
from stoneforge.data_management.preprocessing import DataLoader

In [2]:
# Loading a tab-separated values file with specified standard

tabr = DataLoader(r"../stoneforge/datasets/teste_tsv.tsv", filetype='tabr', sep="\t", std="US")
data_tabr, units_tabr = tabr.dataframe(tabr.data_obj.data)
data_tabr

C:\Users\mario\Documents\GitHub\stoneforge\stoneforge\io\tabr.py:61: UserWarning: Skipping line 21: expected 7 fields, got 6
  warnings.warn(
C:\Users\mario\Documents\GitHub\stoneforge\stoneforge\io\tabr.py:61: UserWarning: Skipping line 22: expected 7 fields, got 6
  warnings.warn(
C:\Users\mario\Documents\GitHub\stoneforge\stoneforge\io\tabr.py:61: UserWarning: Skipping line 26: expected 7 fields, got 6
  warnings.warn(


,PROF,ρB,NPHI,ILD30,ILD,42,�
0,1000,2.65,0.32,"10.000,00",10000,-999.0,ARENITO
1,10003,2.66,0.33,"10.001,05",10001,-999.0,ARENITO
2,10006,2.67,0.34,"10,000.56",10002,-999.0,arenito
3,10009,2.68,0.35,2003,10003,-999.0,folhelho
4,10012,2.69,0.36,-999.000,10004,-999.0,57
5,10015,2.70,0.37,-999.000,10005,-999.0,57
6,10018,2.71,0.38,-999.000,10006,-999.0,57
7,10021,2.72,0.39,-999.000,10007,-999.0,57
8,10024,2.73,0.40,-999.000,10008,-999.0,57
9,10027,2.74,0.41,-999.000,10009,-999.0,slurry]


In [3]:
# Las2 example usage

las2 = DataLoader(r"../stoneforge/datasets/DP1.las")
print('header itens:',las2.data_obj.header.keys())
data_las2, units_las2 = las2.dataframe(las2.data_obj.data)
data_las2

filetype '.las' assumed to be LAS2, trying to parse as LAS2...
LAS2 parsing successful.
header itens: dict_keys(['version', 'well', 'curve', 'parameter', 'other'])


,DEPT,SP,ILD,ILM,LL8,GR,CALI,DRHO,RHOB,NPHI,DT
0,100.0,-999.0,-999.0,-999.0,-999.0,25.1383,-999.0,-999.0,-999.0,-999.0,-999.0
1,100.5,-999.0,-999.0,-999.0,-999.0,24.7342,-999.0,-999.0,-999.0,-999.0,-999.0
2,101.0,-999.0,-999.0,-999.0,-999.0,24.9611,-999.0,-999.0,-999.0,-999.0,-999.0
3,101.5,-999.0,-999.0,-999.0,-999.0,25.1880,-999.0,-999.0,-999.0,-999.0,-999.0
4,102.0,-999.0,-999.0,-999.0,-999.0,25.4148,-999.0,-999.0,-999.0,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...
15688,7944.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0
15689,7944.5,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0
15690,7945.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0
15691,7945.5,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0


In [4]:
# Las3 example usage


las3 = DataLoader(r"../stoneforge/datasets/example_las3.las")
print(las3.data_obj.tables)
las3.data_obj.force_association()
print(las3.data_obj.tables)
las3.data_obj.data['Log_Data']
data_las3, units_las3 = las3.dataframe(las3.data_obj.data['Log_Data'])
data_las3

filetype '.las' assumed to be LAS2, trying to parse as LAS2...
Failed to parse as LAS2, trying LAS3
LAS3 parsing successful.
['VERSION INFORMATION', 'Well Information', 'Log_Parameter', 'Log_Definition', 'Drilling_Definition', 'Drilling_Data | Drilling_Definition', 'Core_Definition', 'Core_Data[1] | Core_Definition', 'Core_Data[2] | Core_Definition', 'Inclinometry_Definition', 'Inclinometry_Data | Inclinometry_Definition', 'Test_Definition', 'Test_Data | Test_Definition', 'TOPS_Definition', 'TOPS_Data | TOPS_Definition', 'Perforations_Definition', 'Perforations_Data | Perforations_Definition', 'Log_Data | Log_Definition']
['VERSION INFORMATION', 'Well Information', 'Log_Parameter', 'Core_Definition', 'Core_Data[1] | Core_Definition', 'Core_Data[2] | Core_Definition', 'Log_Data', 'Drilling_Data', 'Inclinometry_Data', 'Test_Data', 'TOPS_Data', 'Perforations_Data']


C:\Users\mario\Documents\GitHub\stoneforge\stoneforge\io\las3.py:123: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  main_data = main_data.to_dict(orient="list")


,DEPT,DT,RHOB,NPHI,SFLU,SFLA,ILM,ILD,YME,CDES,NMR[1],NMR[2],NMR[3],NMR[4],NMR[5]
0,1670.000,123.45,2550.0,0.45,123.45,123.45,110.2,105.6,1.450000e+12,DOLOMITE WI/VUGS,10.0,12.0,14.0,18.0,13.0
1,1669.875,123.45,2550.0,0.45,123.45,123.45,110.2,105.6,1.470000e+12,LIMESTOVE,12.0,15.0,21.0,35.0,25.0
2,1669.750,123.45,2550.0,0.45,123.45,123.45,110.2,105.6,2.850000e+12,LOST INTERVAL,18.0,25.0,10.0,8.0,17.0


In [5]:
# DLIS example usage

dlis = DataLoader(r"../stoneforge/datasets/DSDP_leg_96_hole_616_96_processed_data.dlis")
data_s = dlis.data_obj.mnemonic_search(mnemonics_list=['CALI' ,'GR', 'RHOB', 'NPHI', 'DT', 'DTCO', 'ILD', 'DTS', 'DCAL', 'SP'])
data_dlis, units_dlis = dlis.dataframe(data_s['FDC/CNL/GR_main']['B59067'])
data_dlis

Trying to parse as DLIS data file due to '.dlis' extention ...
DLIS parsing successful.


,RHOB,GR,CALI,NPHI
0,2.089844,133.250000,7.855561,61.279297
1,2.089844,133.250000,7.855561,61.279297
2,2.089844,133.250000,7.855561,61.279297
3,2.089844,133.250000,7.855561,61.279297
4,2.089844,133.250000,7.855561,61.279297
...,...,...,...,...
2469,1.119141,6.449219,7.917077,98.828125
2470,1.106445,5.375000,7.917077,108.496094
2471,1.120117,3.765625,7.898622,96.728516
2472,1.113281,3.751953,7.923228,106.640625
